# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)

from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [6]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [7]:
df['combined_text'] = df.headline + ' ' + df.short_description
data = df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [8]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [9]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [10]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55562319, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [11]:
# The word vectors themselves are stored inside of model.wv --> let's store this inside of a variable
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [12]:
wv.most_similar('Texas')

[('Ohio', 0.8151410818099976),
 ('Arkansas', 0.8113201856613159),
 ('Pennsylvania', 0.8057567477226257),
 ('Maryland', 0.8042000532150269),
 ('Louisiana', 0.7981703281402588),
 ('Oklahoma', 0.7913118600845337),
 ('Illinois', 0.7912087440490723),
 ('Georgia', 0.7910197377204895),
 ('Arizona', 0.7891300320625305),
 ('Oregon', 0.779640257358551)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [13]:
wv.most_similar(negative='Texas')

[('maitre', 0.4501088261604309),
 ('encapsulate', 0.3837030529975891),
 ('Telecommunication', 0.38329142332077026),
 ('Trashbag', 0.37744688987731934),
 ('Strategist™', 0.3644898533821106),
 ('Un-Blind', 0.36405420303344727),
 ('Overlapping', 0.3627282381057739),
 ('Doctoral', 0.36124736070632935),
 ('Trump-Voting', 0.36059799790382385),
 ('Animal-rescue', 0.35532137751579285)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [14]:
wv['Texas']

array([ 7.4695569e-01,  3.0095615e+00, -9.9813014e-02,  2.5513417e-01,
       -4.9830484e-01,  2.7034781e+00, -1.7261866e+00,  1.9259686e+00,
        2.8099445e-01,  5.1053092e-03,  9.7506875e-01,  3.5474086e+00,
       -2.3671081e+00, -1.7841119e+00, -2.7373433e+00,  9.0789104e-01,
       -1.9859090e+00, -2.1673694e+00, -1.7128562e+00, -8.8785630e-01,
       -6.6615415e-01,  7.5748599e-01, -9.7271544e-01, -1.0754013e+00,
        9.8467559e-01, -1.4287685e-01,  1.3264699e+00, -2.9814976e-01,
        1.8214462e+00, -6.6398084e-01,  6.0605514e-01,  3.9935622e-01,
       -2.3742337e+00, -1.4838449e+00,  1.0947549e+00, -2.2611982e-01,
       -7.9734153e-01,  9.7070318e-01, -1.3556340e+00,  4.2685488e-01,
       -5.6844413e-01,  8.2754999e-02,  1.1606560e+00, -6.5679640e-01,
        9.5409197e-01, -3.2794246e-01,  1.6471508e+00,  4.2904489e-02,
       -3.9640647e-01, -1.9535068e+00, -7.2899133e-01, -1.9261050e+00,
        1.5731614e+00, -4.9180162e-01, -1.2628498e+00, -2.5026804e-01,
      

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [15]:
wv.vectors

array([[ 2.05183554e+00, -4.65381235e-01, -4.10836011e-01, ...,
         2.90150195e-01, -3.50547619e-02, -7.48781025e-01],
       [ 2.12764978e-01, -5.39528668e-01,  1.05815816e+00, ...,
        -1.97910643e+00, -2.17442572e-01, -2.92494833e-01],
       [ 7.03206897e-01,  1.83227733e-01, -2.13698983e+00, ...,
        -1.52916634e+00, -1.59030810e-01, -1.84992003e+00],
       ...,
       [ 1.59696825e-02,  3.72246206e-02,  3.22466940e-02, ...,
        -3.86523232e-02, -6.80352300e-02, -1.22067670e-03],
       [ 6.16874658e-02, -2.05937400e-02, -2.01448370e-02, ...,
         1.48704671e-03, -3.92606296e-02, -5.46090789e-02],
       [-1.56458337e-02,  5.18809520e-02,  4.40116078e-02, ...,
        -1.00864634e-01, -8.08835253e-02,  5.52311279e-02]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [16]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('princess', 0.6215994358062744),
 ('revival', 0.5930920839309692),
 ('queen', 0.5899044275283813),
 ('crown', 0.5614991188049316),
 ('flower', 0.5453428030014038),
 ('brunette', 0.5450067520141602),
 ('monster', 0.5387164354324341),
 ('symbol', 0.538409948348999),
 ('Queen', 0.5373451709747314),
 ('craze', 0.5316087007522583)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!